# Parkinson's Disease Classifier

A logistic regression classifier built from scratch which is used to classify whether a person is suffering from Parkinson's disease or the person is healthy.

In [1]:
#Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize
from sklearn.model_selection import train_test_split

In [232]:
parkinsons_data = pd.read_csv('parkinsons.data', sep = ',')
parkinsons_data

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,phon_R01_S50_2,174.188,230.978,94.261,0.00459,0.00003,0.00263,0.00259,0.00790,0.04087,...,0.07008,0.02764,19.517,0,0.448439,0.657899,-6.538586,0.121952,2.657476,0.133050
191,phon_R01_S50_3,209.516,253.017,89.488,0.00564,0.00003,0.00331,0.00292,0.00994,0.02751,...,0.04812,0.01810,19.147,0,0.431674,0.683244,-6.195325,0.129303,2.784312,0.168895
192,phon_R01_S50_4,174.688,240.005,74.287,0.01360,0.00008,0.00624,0.00564,0.01873,0.02308,...,0.03804,0.10715,17.883,0,0.407567,0.655683,-6.787197,0.158453,2.679772,0.131728
193,phon_R01_S50_5,198.764,396.961,74.904,0.00740,0.00004,0.00370,0.00390,0.01109,0.02296,...,0.03794,0.07223,19.020,0,0.451221,0.643956,-6.744577,0.207454,2.138608,0.123306


In [233]:
# Converting dataframes into NumPy arrays and removing status column to store in 
X_parkinsons = np.array(parkinsons_data.drop(['status'], axis = 1))[:, 1:]
y_parkinsons = np.array(parkinsons_data['status'])
X_parkinsons = X_parkinsons.astype('float64')

In [268]:
X_train, X_val, y_train, y_val = train_test_split(X_parkinsons, y_parkinsons, test_size = 0.2, random_state = 4)

In [300]:
np.count_nonzero(y_parkinsons == 1)

147

##### Feature Normalization Function

In [418]:
def FeatureNormalize(X):
    
    X_norm = X.copy()
    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    
    X_norm = (X_norm - mu)/sigma
    
    return X_norm

#### Sigmoid Function for Logistic Regression

In [235]:
def sigmoid(z):
    
    sigmoid_val = 1 / (1 + np.exp(-z))
    
    return sigmoid_val

##### Cost Function and Gradient for Logistic Regression

In [346]:
def CostFunction(theta, X, y, lambda_):
    
    m = y.size
    
    X = np.concatenate([np.ones((m,1)), X], axis = 1)
    J = 0
    gradient = np.zeros(X.shape[1])
    
    # Computing the cost function and the gradient
    
    h = sigmoid(np.matmul(X, theta.T))
    y_term_1 = y*np.log(h)
    y_term_2 = (1-y)*(np.log(1-h))
    
    J_unreg =  - np.mean(y_term_1 + y_term_2)
    
    J = J_unreg + ((lambda_ * np.sum(np.square(theta[1:]))) / ( 2 * m))
    
    gradient[0] = (h-y).dot(X)[0] / m
    gradient[1:] = ((h-y).dot(X)[1:] + (lambda_ * theta[1:])) / m
    
    return J, gradient

##### Function for predicting the result

In [428]:
def predict(X, theta):
    
    X = np.concatenate([np.ones((X.shape[0],1)), X], axis = 1)
    
    h = sigmoid(np.matmul(X, theta.T))
    y = np.zeros(X.shape[0])
    
    pos = h >= 0.5
    neg = h < 0.5
    
    y[pos] = 1
    y[neg] = 0
    
    return y

##### Function for model accuracy

In [238]:
def model_accuracy(y_pred, y_val):
    
    percent = 0 
    m = y_pred.shape[0]
    
    for i in range(m):
        if y_pred[i] == y_val[i]:
            percent = percent + 1
        else:
            pass
        
    percent = (percent/m) * 100
    
    return percent

##### Function for predicting best value of regularization parameter (optimize.minimize)

In [289]:
def bestlambda(X, y, initial_theta, X_val, y_val):
    
    lambda_ = np.array([0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100, 300])
    options = {'maxiter' : 100}    
    
    percent_accurate = np.zeros(lambda_.shape[0])
    
    for i in range(len(lambda_)):
        
        result = optimize.minimize(CostFunction,
                           initial_theta,
                           (X, y, lambda_[i]),
                           jac = True,
                           method = 'TNC', 
                           options = options)
        
        #Storing values of the trained parameters
        theta_trained  = result.x

        #Predicting results for the cross validation/test set
        y_pred = predict(X_val, theta_trained)

        #Checking how well the model generalizes
        percent_accurate[i] = model_accuracy(y_pred, y_val)
        
    best_accuracy_index = np.argmax(percent_accurate)
    
    lambda_optimal = lambda_[best_accuracy_index]
    
    return lambda_optimal

##### Function for gradient descent

In [412]:
def GradientDescent(initial_theta, num_iters, X, y, m, lambda_ , alpha = 0.1):
    
    theta_trained = initial_theta.copy()
    X = np.concatenate([np.ones((X.shape[0],1)), X], axis = 1)
    
    for i in range(num_iters):
        
        h = sigmoid(np.matmul(X, theta_trained.T))
        term = (h-y).dot(X)
        term_2 = lambda_ * theta_trained[1:]
        
        theta_trained[0] = theta_trained[0] - (alpha * term[0] / m)
        theta_trained[1:] = theta_trained[1:] - ( alpha * (term[1:] + term_2) /m)
        
    return theta_trained

##### Function for choosing best value of learning rate and regularization parameter (Gradient Descent)

In [475]:
def best_alpha_lambda(initial_theta, num_iters, X, y, X_val, y_val, m):
    
    lambda_ = np.array([0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100, 300])
    alpha = np.array([0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1])
    
    percent_accurate_gd = np.zeros((lambda_.shape[0], alpha.shape[0]))
    
    for i in range(lambda_.shape[0]):
        for j in range(alpha.shape[0]):
            
            theta_trained = GradientDescent(initial_theta, num_iters, FeatureNormalize(X), y, m, lambda_[i],  alpha[j])
            
            y_pred_gd = predict(FeatureNormalize(X_val), theta_trained)
            
            percent_accurate_gd[i, j] = model_accuracy(y_pred_gd, y_val)

    best_accuracy_index = np.unravel_index(percent_accurate_gd.argmax(), percent_accurate_gd.shape)
 
    
    lambda_optimal = lambda_[best_accuracy_index[0]]
    alpha_optimal = alpha[best_accuracy_index[1]]
    
    return lambda_optimal, alpha_optimal

##### Training the Logistic Regression algorithm using optimize.minimize

In [438]:
#Initializing initial parameters as zero
n = X_parkinsons.shape[1]
initial_theta = np.zeros(n+1)

#Initializing value of regularization parameter
lambda_ = bestlambda(X_train, y_train, initial_theta, X_val, y_val)

#Training the algorithm on the training dataset
options = {'maxiter' : 100}

result = optimize.minimize(CostFunction,
                           initial_theta,
                           (X_train, y_train, lambda_),
                           jac = True,
                           method = 'TNC', 
                           options = options)

#Storing values of the trained parameters
theta_trained  = result.x

#Predicting results for the cross validation/test set
y_pred = predict(X_val, theta_trained)

#Checking how well the model generalizes
percent_accurate = model_accuracy(y_pred, y_val)
percent_accurate

84.61538461538461

##### Training the logistic regression algorithm using Gradient Descent

In [477]:
n = X_parkinsons.shape[1]
initial_theta = np.zeros(n+1)

#Entering the number of iterations to be done
num_iters = 100

m = y_train.size

#Initializing learning rate value and regularization parameter
lambda_, alpha = best_alpha_lambda(initial_theta, num_iters, X_train, y_train, X_val, y_val, m)

# J, gradient = CostFunction(initial_theta, X_train, y_train, lambda_)

#Training the model using Gradient Descent
theta_trained = GradientDescent(initial_theta, num_iters, FeatureNormalize(X_train), y_train, m, lambda_,  alpha)

y_pred_gd = predict(FeatureNormalize(X_val), theta_trained)

#Checking how well the Gradient Descent algorithm generalizes
percent_accurate_gd = model_accuracy(y_pred_gd, y_val)
percent_accurate_gd

87.17948717948718